In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate,GlobalMaxPooling1D
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU, BatchNormalization
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [2]:
# define network parameters
max_features = 128
maxlen = 512

In [3]:
#read csv
dict_label = {'Usable directly numeric':0, 'Usable with extraction':1, 'Usable with Extration': 1, 'Usable with extraction ':1, 'Usable directly categorical':2, 'Unusable':3, 'Context_specific':4, 'Usable directly categorical ':2}
data = pd.read_csv('data_for_ML_num.csv')
data['y_act'] = [dict_label[i] for i in data['y_act']]
y = data.loc[:,['y_act']]
data_LSTM = pd.concat([data['Attribute_name'], data['sample_1'], data['sample_2'], data['sample_3'], data['sample_4'], data['sample_5']], axis =1)


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data = data.rename(columns={'Num of nans': 'Num_of_nans', 'num of dist_val': 'num_of_dist_val'})

# data['Num_of_nans'] = [float(data['Num_of_nans'][i])/float(data['Total_val'][i]) for i in data.index]
# data['num_of_dist_val'] = [float(data['num_of_dist_val'][i])/float(data['Total_val'][i]) for i in data.index]

data1 = data[['Num_of_nans', 'max_val', 'mean', 'min_val', 'num_of_dist_val','std_dev','castability','extractability', 'len_val','Total_val']]
data1 = data1.fillna(0)

data1['Num_of_nans'] = [data1['Num_of_nans'][i]*100/data1['Total_val'][i] for i in data1.index]
data1['num_of_dist_val'] = [(data1['num_of_dist_val'][i]*1.0)/data1['Total_val'][i] for i in data1.index]

print('hghj')
data1 = data1.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev'})
data1.loc[data1['scaled_min_val'] > 10000, 'scaled_min_val'] = 10000
data1.loc[data1['scaled_min_val'] < -10000, 'scaled_min_val'] = -10000
data1.loc[data1['scaled_max_val'] > 10000, 'scaled_max_val'] = 10000
data1.loc[data1['scaled_max_val'] < -10000, 'scaled_max_val'] = -10000
data1.loc[data1['scaled_mean'] > 10000, 'scaled_mean'] = 10000
data1.loc[data1['scaled_mean'] < -10000, 'scaled_mean'] = -10000
data1.loc[data1['scaled_std_dev'] > 10000, 'scaled_std_dev'] = 10000
data1.loc[data1['scaled_std_dev'] < -10000, 'scaled_std_dev'] = -10000
column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev','num_of_dist_val','Num_of_nans','Total_val']
x = data1[column_names_to_normalize].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data1.index)
data1[column_names_to_normalize] = df_temp

data1.Num_of_nans = data1.Num_of_nans.astype(float)
data1.num_of_dist_val = data1.num_of_dist_val.astype(float)
data1.castability = data1.castability.astype(float)
data1.extractability = data1.extractability.astype(float)
y.y_act = y.y_act.astype(float)


print('hghj')

data1 = data1[['Num_of_nans', 'scaled_max_val', 'scaled_mean', 'scaled_min_val', 'num_of_dist_val','scaled_std_dev','castability','extractability', 'len_val','Total_val']]


hghj
hghj


In [5]:
# X_train, X_test,y_train,y_test = train_test_split(data2,y, test_size=0.2,random_state=100)
# key_name = data['Attribute_name']
# atr_train,atr_test = train_test_split(key_name, test_size=0.2,random_state=100)

# # print(X_train)
# X_train.to_csv('xtrain.csv')
# atr_train.to_csv('attrain.csv')

# new_df  = X_train
# X_train = X_train.reset_index(drop=True)
# atr_train.reset_index(inplace=True,drop=True)

# X_train.to_csv('xtrain1.csv')
# X_test = X_test.reset_index(drop=True)
# y_train = y_train.reset_index(drop=True)
# y_test = y_test.reset_index(drop=True)
# atr_train.reset_index(inplace=True,drop=True)
# atr_test.reset_index(inplace=True,drop=True)

# lendf = len(X_train)
# print(len(X_train))
# # print(X_train)

# X_trainv = X_train.values
# y_trainv = y_train.values
# atr_trainv = atr_train.values


# print(X_trainv)
# print(atr_trainv)

# # i=0
# # for index,row in X_train.iterrows():
# #     print(index)
# #     if i==3:
# #         break
# #     print(row)
# #     new_df.loc[lendf+i] = row
# #     i=i+1
    
# # new_df = new_df.reset_index(drop=True)
# # print(new_df)

# i=0
# for x in X_trainv:
#     X_trainv = np.vstack((X_trainv,X_trainv[i]))
#     y_trainv = np.vstack((y_trainv,y_trainv[i]))    
#     atr_trainv = np.hstack((atr_trainv,atr_trainv[i]))
#     i=i+1
    
# print(X_trainv)
# print(len(X_trainv))
# print(atr_trainv)
# print(len(atr_trainv))

# X_train = pd.DataFrame(X_trainv,columns = ['Num_of_nans', 'scaled_max_val', 'scaled_mean', 'scaled_min_val', 'num_of_dist_val','scaled_std_dev','castability','extractability', 'len_val'])
# y_train = pd.DataFrame(y_trainv, columns =['y_act'])
# print(X_train)
# print(y_train)
# # atr_train = pd.DataFrame(atr_trainv)
# # print(atr_train)

In [40]:
arr = data['Attribute_name'].values
arr1 = data['sample_1'].values
arr1 = [str(x) for x in arr1]
arr2 = data['sample_2'].values
arr2 = [str(x) for x in arr2]

vectorizer = CountVectorizer(ngram_range=(2,2),analyzer='char')
X = vectorizer.fit_transform(arr)
X1 = vectorizer.fit_transform(arr1)
X2 = vectorizer.fit_transform(arr2)
print(len(vectorizer.get_feature_names()))
# print(X.toarray())

# data1.to_csv('before.csv')

tempdf = pd.DataFrame(X.toarray())
tempdf1 = pd.DataFrame(X1.toarray())
tempdf2 = pd.DataFrame(X2.toarray())

data2 = pd.concat([data1,tempdf,tempdf1,tempdf2], axis=1, sort=False)

1927


In [ ]:
X_train, X_test,y_train,y_test = train_test_split(data2,y, test_size=0.2,random_state=100)

key_name = data['Attribute_name']
atr_train,atr_test = train_test_split(key_name, test_size=0.2,random_state=100)


X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
# atr_train.reset_index(inplace=True,drop=True)
# atr_test.reset_index(inplace=True,drop=True)

print(len(X_train))
# print(atr_train)
list_sentences_train = atr_train.values
# list_sentences_train = atr_trainv
list_sentences_test = atr_test.values
# print(list_sentences_train)

structured_data_train = X_train 
structured_data_test = X_test

print(len(structured_data_train))
print(len(list(structured_data_train.loc[2])))

structured_input_train = []
for i in range(len(structured_data_train)):
    if i%100 == 0:
        print(i)
    structured_input_train.append(list(structured_data_train.loc[i]))
structured_input_train = np.array(structured_input_train).reshape(len(structured_data_train),len(structured_data_train.keys()))    

structured_input_test = []
for i in range(len(structured_data_train)):
    structured_input_test.append(list(structured_data_train.loc[i]))
structured_input_test = np.array(structured_input_test).reshape(len(structured_data_train),len(structured_data_train.keys()))    

print(len(structured_data_train))
print(len(list_sentences_train))

7400
7400
5122
0
100
200
300


In [12]:
key_name = data['sample_1']
samp1_train,samp1_test = train_test_split(key_name, test_size=0.2,random_state=100)
samp1_train.reset_index(inplace=True,drop=True)
samp1_test.reset_index(inplace=True,drop=True)


key_name = data['sample_2']
samp2_train,samp2_test = train_test_split(key_name, test_size=0.2,random_state=100)
samp2_train.reset_index(inplace=True,drop=True)
samp2_test.reset_index(inplace=True,drop=True)

In [13]:

list_sentences_train = atr_train.values
list_sentences_test = atr_test.values

print(len(list_sentences_train))

# X_train.sample_1 = X_train.sample_1.astype(str)
# X_test.sample_1 = X_test.sample_1.astype(str)

list_sentences_train1 = samp1_train.values
list_sentences_test1 = samp1_test.values

print(list_sentences_train1)

# X_train.sample_2 = X_train.sample_2.astype(str)
# X_test.sample_2 = X_test.sample_2.astype(str)

list_sentences_train2 = samp2_train.values
list_sentences_test2 = samp2_test.values

print(list_sentences_train2)

for i in range(len(list_sentences_train)):
    list_sentences_train[i] = str(list_sentences_train[i])
    list_sentences_train1[i] = str(list_sentences_train1[i])
    list_sentences_train2[i] = str(list_sentences_train2[i])
    
for i in range(len(list_sentences_test)):
    list_sentences_test[i] = str(list_sentences_test[i])
    list_sentences_test1[i] = str(list_sentences_test1[i])
    list_sentences_test2[i] = str(list_sentences_test2[i])    

print(len(list_sentences_train))

7400
['#NULL!' '0' '268' ... 'Hate' '-0.101' '2']
['0' '78' '110' ... nan '0.171' '5']
7400


In [14]:
tokenizer = keras_text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(list(list_sentences_train))
# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=maxlen)
# test data
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = keras_seq.pad_sequences(list_tokenized_test, maxlen=maxlen)


tokenizer1 = keras_text.Tokenizer(char_level = True)
tokenizer1.fit_on_texts(list(list_sentences_train1))
# train data
list_tokenized_train1 = tokenizer.texts_to_sequences(list_sentences_train1)
X_t1 = keras_seq.pad_sequences(list_tokenized_train1, maxlen=maxlen)
# test data
list_tokenized_test1 = tokenizer.texts_to_sequences(list_sentences_test1)
X_te1 = keras_seq.pad_sequences(list_tokenized_test1, maxlen=maxlen)


# tokenizer2 = keras_text.Tokenizer(char_level = True)
# tokenizer2.fit_on_texts(list(list_sentences_train1))
# # train data
# list_tokenized_train2 = tokenizer.texts_to_sequences(list_sentences_train2)
# X_t2 = keras_seq.pad_sequences(list_tokenized_train2, maxlen=maxlen)
# # test data
# list_tokenized_test2 = tokenizer.texts_to_sequences(list_sentences_test2)
# X_te2 = keras_seq.pad_sequences(list_tokenized_test2, maxlen=maxlen)

# print(tokenizer)

In [35]:
def build_model(numfilters,embed_size):
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer.word_counts)+1, output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.5)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(numfilters, kernel_size = 3, activation = 'tanh')(x)
        numfilters = numfilters*2
    
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    xy = concatenate(out_conv, axis = -1)  
    print(xy.shape)
    #########################################################
    inp1 = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer1.word_counts)+1, output_dim = embed_size)(inp1)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(16*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    x1 = concatenate(out_conv, axis = -1)
    print(x1.shape)
    
    Str_input = Input(shape=(3195,))
    layersfin = keras.layers.concatenate([xy,Str_input])
    print(layersfin.shape)
    x = Dense(500, activation='relu')(layersfin)
    x = Dropout(0.75)(x)
    x = Dense(500, activation='relu')(x)
    x = Dropout(0.75)(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(inputs=[inp,Str_input], outputs=[x])
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# model = build_model()
# model.summary()

In [36]:
batch_size = 32 # large enough that some other labels come in
epochs = 25

file_path="best_weights.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=20)

callbacks_list = [checkpoint, early] #early
# history = model.fit([X_t,X_t1,structured_data_train], to_categorical(y_train),
#                     validation_data=([X_te,X_te1,structured_data_test], to_categorical(y_test)),
#                     batch_size=batch_size, epochs=epochs, shuffle = True, callbacks=callbacks_list)

# # model = build_model()
# # model.summary()


In [37]:
print(X_t)
print(X_t.shape)
# print(y_train[1851:])
print(len(y_train))
y_train = y_train.values
structured_data_train = structured_data_train.values



[[ 0  0  0 ...  2 11  2]
 [ 0  0  0 ... 37 22 39]
 [ 0  0  0 ... 15  5 23]
 ...
 [ 0  0  0 ... 18  5  8]
 [ 0  0  0 ... 28 16 33]
 [ 0  0  0 ...  4  1  2]]
(7400, 512)
7400


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [38]:
# n_filters_grid = [16,32,64,128]
# embed_size = [32,64,128,512]

batch_size = 32 # large enough that some other labels come in
epochs = 25

k = 5
kf = KFold(n_splits=k)
n_filters_grid = [32]
embed_size = [256]
models = []

avgsc_lst,avgsc_val_lst,avgsc_train_lst = [],[],[]
avgsc,avgsc_val,avgsc_train = 0,0,0
i=0
for train_index, test_index in kf.split(X_t):
    file_path= 'best_weights'+str(i)+'.h5'
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    early = EarlyStopping(monitor="val_acc", mode="max", patience=20)

    callbacks_list = [checkpoint, early] #early
        
    print('\n')
    X_train_cur, X_test_cur = X_t[train_index], X_t[test_index]
    y_train_cur, y_test_cur = y_train[train_index], y_train[test_index]
    structured_data_train_cur, structured_data_test_cur = structured_data_train[train_index],structured_data_train[test_index]
    
    X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)
    structured_data_train_train,structured_data_val = train_test_split(structured_data_train_cur, test_size=0.25,random_state=100)
    
    
    print(X_train_cur)
    
    print(y_train_train.shape)
    print(y_val.shape)
    print(structured_data_train_train.shape)
    print(structured_data_val.shape)
    
    bestPerformingModel = build_model(10,5)
    bestscore = 0
    for ne in n_filters_grid:
        for md in embed_size:
#             clf = RandomForestClassifier(n_estimators=ne,max_depth=md)
            clf = build_model(ne,md)
            history = clf.fit([X_train_cur,structured_data_train_cur], to_categorical(y_train_cur),
                    validation_data=([X_test_cur,structured_data_test_cur], to_categorical(y_test_cur)),
                    batch_size=batch_size, epochs=epochs, shuffle = True, callbacks=callbacks_list)
        
            sc = clf.evaluate([X_test_cur,structured_data_test_cur],to_categorical(y_test_cur))
            bestPerformingModel = clf
            if bestscore < sc:
                bestscore = sc
                bestPerformingModel = clf

#     clf.load_weights('best_weights'+str(i)+'.h5')
    print(sc)
    avgsc_val_lst.append(sc[1])
    
    loss, bscr_train = bestPerformingModel.evaluate([X_train_cur,structured_data_train_cur],to_categorical(y_train_cur))
    print(loss, bscr_train)
    loss, bscr_val = bestPerformingModel.evaluate([X_test_cur,structured_data_test_cur],to_categorical(y_test_cur))
    print(loss, bscr_val)    
    loss, bscr = bestPerformingModel.evaluate([X_te,structured_data_test],to_categorical(y_test))
    print(loss, bscr)
    
    models.append(clf)
    
    avgsc = avgsc + bscr
    
    avgsc_train = avgsc_train + bscr_train
    avgsc_val = avgsc_val + bscr_val
    avgsc_lst.append(bscr)
    avgsc_train_lst.append(bscr_train)
    print('The training accuracy is:')
    print(bscr_train)
    print('The validation accuracy is:')
    print(bscr_val)    
    print('The test accuracy is:')    
    print(bscr)
    print('\n')
    i=i+1



[[ 0  0  0 ...  3 11 10]
 [ 0  0  0 ... 22  6 33]
 [ 0  0  0 ...  4  1  2]
 ...
 [ 0  0  0 ... 18  5  8]
 [ 0  0  0 ... 28 16 33]
 [ 0  0  0 ...  4  1  2]]
(4440, 1)
(1480, 1)
(4440, 3195)
(1480, 3195)
(?, 40)
(?, 64)
(?, 3235)
(?, 128)
(?, 64)
(?, 3323)
Train on 5920 samples, validate on 1480 samples
Epoch 1/25
5920/5920 [==============================] - 3s 530us/step - loss: 1.1139 - acc: 0.5789 - val_loss: 0.6752 - val_acc: 0.7689

Epoch 00001: val_acc improved from -inf to 0.76892, saving model to best_weights0.h5
Epoch 2/25
5920/5920 [==============================] - 1s 232us/step - loss: 0.7043 - acc: 0.7517 - val_loss: 0.5037 - val_acc: 0.8243

Epoch 00002: val_acc improved from 0.76892 to 0.82432, saving model to best_weights0.h5
Epoch 3/25
5920/5920 [==============================] - 1s 232us/step - loss: 0.5555 - acc: 0.8096 - val_loss: 0.4607 - val_acc: 0.8426

Epoch 00003: val_acc improved from 0.82432 to 0.84257, saving model to best_weights0.h5
Epoch 4/25
5920/5920 [=

In [39]:
print(models)
print(avgsc_train_lst)
print(avgsc_val_lst)
print(avgsc_lst)

# i = avgsc_lst.index(max(avgsc_lst))
# bestPerformingModel = models[i]

print(avgsc_train/k)
print(avgsc_val/k)
print(avgsc/k)


bestPerformingModel

loss, bscr = bestPerformingModel.evaluate([X_te,structured_data_test],to_categorical(y_test))
y_pred = bestPerformingModel.predict([X_te,structured_data_test])
print(y_pred)
y_pred = [np.argmax(i) for i in y_pred]
cm = confusion_matrix(y_test,y_pred)
print(cm)


[<keras.engine.training.Model object at 0x7f844c19bad0>, <keras.engine.training.Model object at 0x7f81940d4710>, <keras.engine.training.Model object at 0x7f8b2e470dd0>, <keras.engine.training.Model object at 0x7f8b2d9088d0>, <keras.engine.training.Model object at 0x7f8b2cc00710>]
[0.9841216216216216, 0.9871621621621621, 0.9841216216216216, 0.9863175675675676, 0.9853040540540541]
[0.8783783783783784, 0.8655405405405405, 0.8635135135135135, 0.8844594594594595, 0.8682432432432432]
[0.8854673150614878, 0.8800648299183219, 0.8800648302403351, 0.8795245811785961, 0.8903295516903371]
0.9854054054054056
0.872027027027027
0.8830902216178156
1851/1851 [==============================] - 0s 76us/step
[[1.0000000e+00 5.8397471e-13 4.9001246e-14 4.3336051e-12 1.4921083e-08]
 [1.0000000e+00 1.7303493e-10 1.3577841e-09 5.4521276e-09 2.3144874e-10]
 [5.5389351e-06 1.1416386e-05 1.6488797e-04 1.1002002e-04 9.9970812e-01]
 ...
 [1.8275927e-05 8.9471841e-06 9.9628335e-01 3.4965996e-03 1.9293479e-04]
 [9.9

In [223]:
class0,class1,class2,class3,class4 = 0,0,0,0,0
for x in y_test['y_act']:
    if x==0:
        class0 = class0+1
    if x==1:
        class1 = class1+1        
    if x==2:
        class2 = class2+1
    if x==3:
        class3 = class3+1
    if x==4:
        class4 = class4+1
sm = class0+class1+class2+class3+class4
print(class0*1.0/sm,class1*1.0/sm,class2*1.0/sm,class3*1.0/sm,class4*1.0/sm)
print(class0,class1,class2,class3,class4)

(0.38141545110750946, 0.07725553754727174, 0.23284710967044842, 0.0902215018908698, 0.2182603997839006)
(706, 143, 431, 167, 404)


In [219]:
y_pred_df = pd.DataFrame({'y_pred':y_pred})
# print(y_pred)
dfn = pd.concat([y_pred_df,y_test],axis=1)
dfn.to_csv('dfny.csv')

In [50]:
# 'Usable directly numeric':0
# 'Usable with extraction':1 
# 'Usable directly categorical':2
# 'Unusable':3
# 'Context_specific':4

In [17]:
print(len(X_test))

1851


In [5]:
newdf = pd.read_csv('dfn.csv')
newdf = newdf[newdf['y_act']!=newdf['y_pred']]

In [18]:
print(newdf)
newdf.to_csv('diff_cnn.csv')

      Unnamed: 0                  Attribute_name  Num_of_nans  scaled_max_val  \
19            19                             SSR         99.0       -0.466200   
40            40                       Reference          7.0        2.285627   
60            60                           CHROM          0.0       -0.470610   
61            61                       father_pk          1.0        2.285627   
65            65                     toss_winner          0.0       -0.470610   
70            70                             ibu         41.0       -0.432574   
76            76                      s3p1c6lost         81.0       -0.443048   
78            78                  Variable Label          0.0       -0.470610   
87            87                         Version          0.0       -0.470610   
106          106                   Facility Name         14.0       -0.470610   
119          119                   n.valid_until          0.0       -0.470610   
124          124            